In [134]:
import numpy as np
import sklearn
from sklearn import datasets
import matplotlib as plt
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
import seaborn as sns
import plotly.graph_objects as go
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

In [116]:
def medidorDistEntr(X,y,adi,n_base):
    #---------1-----------#
    X1,X_test,y1,y_test = train_test_split(X, y,stratify=y,test_size=0.2,random_state=42,shuffle=True)
    #--------2------------#
    # Para calcular el porcentaje
    numClass= len(np.unique(y1)) #saber cuantas clases diferentes hay
    numTotal = X1.shape[0]
    porc = ((n_base*numClass*100)/numTotal)/100
    X_Pool,X_Base,y_Pool,y_Base = train_test_split(X1, y1,stratify=y1,test_size=porc,random_state=42,shuffle=True)
    #-------3------------#
    centros=[]
    val, cont = np.unique(y_Base,return_counts=True)
    for i in val:
        ind_i=np.where(y_Base==i)
        suma = sum(X_Base[ind_i])
        centro=suma/cont[i]
        centros.append(centro)
    #-------4-------------#
    entropies = [entropy(softmax(listEuclidean(x,centros))) for x in X_Pool]
    #-------5------------#
    entroMaxs = sorted(entropies,reverse=True)[:adi]
    in_add=[]
    for en in entroMaxs:
        in_add.append(np.where(np.array(entropies) == en)[0][0]) #Para calcular los índices con mayor entropía
        
    X_adicionantes=X_Pool[in_add]
    y_adicionantes=y_Pool[in_add]
    #-----Finalmente añadimos al subconjunto base-----#
    X_res = np.append(X_Base,X_adicionantes,axis=0)
    y_res = np.append(y_Base,y_adicionantes)
    
    return X_res, y_res

In [120]:
def listEuclidean (p1,cen):
    return [euclidean(p1,c) for c in cen]

def euclidean (p1,p2):
    res_e = (p2-p1)**2
    suma = sum(res_e)
    return math.sqrt(suma)

def softmax(distances):
    expon = np.exp(distances)
    suma = np.sum(expon)
    return expon/suma
    
def entropy(softProb):
    op = softProb*(np.log2(softProb))
    return -1*(np.sum(op))

In [283]:
import os
for dirname, _, filenames in os.walk('fashion-mnist'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
train = pd.read_csv('fashion-mnist/fashion-mnist_train.csv')
test = pd.read_csv('fashion-mnist/fashion-mnist_test.csv')

df_train = train.copy()
df_test = test.copy()

X_train = df_train.drop(['label'],axis = 1)
y_train = df_train['label']

X_test = df_test.drop(['label'],axis = 1)
y_test = df_test['label']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /=255.0

fashion-mnist\fashion-mnist_test.csv
fashion-mnist\fashion-mnist_train.csv
fashion-mnist\t10k-images-idx3-ubyte
fashion-mnist\t10k-labels-idx1-ubyte
fashion-mnist\train-images-idx3-ubyte
fashion-mnist\train-labels-idx1-ubyte


In [284]:
X_train=np.array(X_train)
y_train=np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [285]:
X_res,y_res = medidorDistEntr(X_train,y_train,1000,500)

(60000,)

In [286]:
X_res = X_res.reshape((X_res.shape[0], 28,28 ))

In [195]:
X_test = X_test.reshape((X_test.shape[0],28,28))

In [287]:
y_res = to_categorical(y_res)
#y_test = to_categorical(y_test)

In [143]:
from tensorflow.keras.optimizers import SGD

In [159]:
def define_model():
    model = keras.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(10, activation='softmax'))
     # compile model
    opt = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
##################################
#            Medidor             #
##################################

In [288]:
X_res.shape

(6000, 28, 28)

In [289]:
y_res.shape

(6000, 10)

In [292]:
model = define_model()

In [293]:
inicio=time.time()
train = model.fit(X_res, y_res, epochs=15, batch_size=32,validation_data=(X_testf, y_testf))
fin=time.time()

Epoch 1/15
188/188 [==============================] - 3s 13ms/step - loss: 0.7953 - accuracy: 0.7190 - val_loss: 0.5412 - val_accuracy: 0.8042
Epoch 2/15
188/188 [==============================] - 2s 13ms/step - loss: 0.4867 - accuracy: 0.8310 - val_loss: 0.5115 - val_accuracy: 0.8142
Epoch 3/15
188/188 [==============================] - 3s 13ms/step - loss: 0.3931 - accuracy: 0.8592 - val_loss: 0.4532 - val_accuracy: 0.8317
Epoch 4/15
188/188 [==============================] - 3s 14ms/step - loss: 0.3265 - accuracy: 0.8852 - val_loss: 0.4119 - val_accuracy: 0.8558
Epoch 5/15
188/188 [==============================] - 2s 13ms/step - loss: 0.2858 - accuracy: 0.8962 - val_loss: 0.3832 - val_accuracy: 0.8558
Epoch 6/15
188/188 [==============================] - 2s 13ms/step - loss: 0.2518 - accuracy: 0.9123 - val_loss: 0.4101 - val_accuracy: 0.8575
Epoch 7/15
188/188 [==============================] - 2s 13ms/step - loss: 0.2111 - accuracy: 0.9275 - val_loss: 0.3910 - val_accuracy: 0.8683

In [282]:
fin-inicio

36.916704416275024

In [ ]:
######################
#      Base          #
######################

In [222]:
X_train = X_train.reshape((X_train.shape[0], 28,28 ))
y_train = to_categorical(y_train)

In [273]:
X_train.shape

(4800, 28, 28)

In [274]:
y_train.shape

(4800, 10)

In [272]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, stratify=y_train, test_size = 0.92)

In [202]:
X, X_testf, y, y_testf = train_test_split(X_train, y_train, stratify=y_train, test_size = 0.2)

In [275]:
model = define_model()

In [276]:
inicio= time.time()
train = model.fit(X_train, y_train, epochs=15, batch_size=32,validation_data=(X_testf, y_testf))
fin = time.time()

Epoch 1/15
150/150 [==============================] - 3s 14ms/step - loss: 0.7903 - accuracy: 0.7096 - val_loss: 0.5547 - val_accuracy: 0.7975
Epoch 2/15
150/150 [==============================] - 2s 13ms/step - loss: 0.5052 - accuracy: 0.8056 - val_loss: 0.4774 - val_accuracy: 0.8383
Epoch 3/15
150/150 [==============================] - 2s 13ms/step - loss: 0.4304 - accuracy: 0.8429 - val_loss: 0.4538 - val_accuracy: 0.8367
Epoch 4/15
150/150 [==============================] - 2s 13ms/step - loss: 0.3761 - accuracy: 0.8629 - val_loss: 0.4130 - val_accuracy: 0.8617
Epoch 5/15
150/150 [==============================] - 2s 13ms/step - loss: 0.3130 - accuracy: 0.8848 - val_loss: 0.3981 - val_accuracy: 0.8625
Epoch 6/15
150/150 [==============================] - 2s 13ms/step - loss: 0.2853 - accuracy: 0.8990 - val_loss: 0.3880 - val_accuracy: 0.8625
Epoch 7/15
150/150 [==============================] - 2s 15ms/step - loss: 0.2462 - accuracy: 0.9087 - val_loss: 0.3428 - val_accuracy: 0.8842

In [277]:
fin-inicio

31.705026626586914

In [250]:
####################
#     COMPLETO     #
####################

In [262]:
X_train = X_train.reshape((X_train.shape[0], 28,28 ))
y_train = to_categorical(y_train)

In [263]:
X_train.shape

(60000, 28, 28)

In [264]:
y_train.shape

(60000, 10)

In [265]:
X_train_com, X_test_com, y_train_com, y_test_com = train_test_split(X_train, y_train, stratify=y_train, test_size = 0.20)

In [267]:
import time

In [266]:
model = define_model()

In [268]:
inicio=time.time()
train = model.fit(X_train, y_train, epochs=15, batch_size=32,validation_data=(X_test, y_test))
fin=time.time()

Epoch 1/15
1875/1875 [==============================] - 25s 13ms/step - loss: 0.4181 - accuracy: 0.8505 - val_loss: 0.2746 - val_accuracy: 0.9092
Epoch 2/15
1875/1875 [==============================] - 27s 14ms/step - loss: 0.2816 - accuracy: 0.8981 - val_loss: 0.2280 - val_accuracy: 0.9217
Epoch 3/15
1875/1875 [==============================] - 26s 14ms/step - loss: 0.2411 - accuracy: 0.9128 - val_loss: 0.1924 - val_accuracy: 0.9300
Epoch 4/15
1875/1875 [==============================] - 26s 14ms/step - loss: 0.2121 - accuracy: 0.9222 - val_loss: 0.1823 - val_accuracy: 0.9375
Epoch 5/15
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1903 - accuracy: 0.9293 - val_loss: 0.1633 - val_accuracy: 0.9392
Epoch 6/15
1875/1875 [==============================] - 26s 14ms/step - loss: 0.1707 - accuracy: 0.9373 - val_loss: 0.1464 - val_accuracy: 0.9408
Epoch 7/15
1875/1875 [==============================] - 25s 13ms/step - loss: 0.1550 - accuracy: 0.9429 - val_loss: 0.1174 -

In [269]:
fin-inicio

388.7532956600189

In [294]:
######################################################
#                         CNN                        #
######################################################
#  Conjunto   #    prec    #   tiempo    #  Muestras #
######################################################
# Indicador   #    0.88    #   36.9167   # 6000 (5000 + 1000)   #
# Estratifi   #    0.88    #   31.705    #    5000   #
# Completo    #    0.98    #   388.753   #   60000   #
######################################################